In [8]:
!pip install thefuzz[speedup]

import pandas as pd
from thefuzz import process
from google.colab import files

# 📌 1. Leer el archivo con la codificación correcta
df = pd.read_csv("/content/dataset_original.csv", encoding='utf-8')

# 📌 2. Limpiar nombres de columnas (por si tienen espacios)
df.columns = df.columns.str.strip()

# 📌 3. Eliminar filas con valores faltantes en columnas clave
df = df.dropna(subset=['Producto', 'Categoria', 'Cliente', 'Fecha', 'Cantidad', 'Precio', 'Tienda', 'Ciudad'])

# 📌 4. Convertir fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df = df.dropna(subset=['Fecha'])

# 📌 5. Eliminar precios y cantidades inválidas
df = df[df['Precio'] > 0]
df = df[df['Cantidad'] > 0]

# 📌 6. Eliminar duplicados
df = df.drop_duplicates(subset=['ID_Venta'])

# 📌 7. Normalizar columnas de texto
cols_texto = ['Producto', 'Categoria', 'Cliente', 'Tienda', 'Ciudad']
for col in cols_texto:
    df[col] = df[col].astype(str).str.lower().str.strip()

# 📌 8. Lista de productos válidos
productos_validos = ['bota', 'camisa', 'carpeta', 'cuaderno', 'falda', 'mochila', 'monitor', 'mouse', 'pantalla', 'pluma', 'sandalia', 'sueter', 'teclado', 'tenis', 'usb', 'zapato' ]

# 📌 9. Corregir errores con thefuzz
def corregir_producto(palabra):
    mejor_match, score = process.extractOne(palabra, productos_validos)
    if score >= 85:
        return mejor_match
    else:
        return palabra

df['Producto'] = df['Producto'].apply(corregir_producto)

# 📌 10. Capitalizar texto final
for col in cols_texto:
    df[col] = df[col].str.title()

# ✅ 11. Guardar como archivo Excel (.xlsx) para evitar errores de codificación
df.to_excel("/content/ventas_limpio.xlsx", index=False)

# ✅ 12. Descargar el archivo limpio en Excel
files.download("/content/ventas_limpio.xlsx")

print("🎉 Dataset limpio guardado y descargado como Excel (.xlsx)")

/tmp/ipython-input-2611494894.py:17: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 Dataset limpio guardado y descargado como Excel (.xlsx)
